Импорт необходимых библиотек:

In [2]:
import numpy as np
import matplotlib.pylab as plt
from scipy.fftpack import fft, fftfreq, ifft
from datetime import datetime, timedelta

Задание параметров обработки:

In [3]:
sample_rate         = 32000     #Hz

M1_amplifier_dB     = 26.       #dB
M3_amplifier_dB     = 32.       #dB
M1_sensitivity_dB   = -165.     #dB re 1V/µPa
M3_sensitivity_dB   = -170.     #dB re 1V/µPa

station             = 'M1'
                            # ( year, month, day, hour, min, sec, microsecond)
start_time          = datetime( 2014,     6,  20,    9,  40,   0 )
end_time            = datetime( 2014,     6,  20,   10,  00,   0 )

freq_band           = [1000, 2000] #Hz

Чтение файлов и отображение сигнала:

In [20]:
duration = end_time - start_time
files_num = int(np.ceil(duration.seconds / 600))

data_array_V = np.array([0])

def file_name(station, start, end):
    return f'{station}_{start.day:02d}{start.month:02d}{start.hour:02d}{start.minute:02d}-{end.day:02d}{end.month:02d}{end.hour:02d}{end.minute:02d}.dat'

current_time = start_time

samples = np.zeros(duration.seconds * sample_rate * files_num)
data_arrays = []

for i in range(files_num):
    next_time = start_time + timedelta(minutes = 10)
    with open(file_name(station, current_time, next_time), 'r') as fID:
        samples = np.fromfile(fID, np.int16).astype(np.float32)
        data_arrays.append(samples)
    continue_time = next_time

data_array_V = np.concatenate(data_arrays)

positive_mask = data_array_V > 0
negative_mask = data_array_V < 0

constant = (M1_sensitivity_dB + M1_amplifier_dB) * (station == 'M1') + (M3_sensitivity_dB + M3_amplifier_dB) * (station == 'M3')

SPL = np.zeros_like(data_array_V)
SPL[positive_mask] = 20 * np.log10(data_array_V[positive_mask]) - constant
SPL[negative_mask] = -20 * np.log10(np.abs(data_array_V[negative_mask])) + constant

time = np.linspace(start_time, end_time, duration.seconds * sample_rate)

# plt.plot(time, SPL)
# plt.xlabel('time [s]')
# plt.ylabel('SPL (???) [dB re 1 µPa]')

Преобразование Фурье, вырезание частоты и отрисовка спектров:

In [21]:
ft = fft(SPL)
freq = fftfreq(SPL.size, 1/sample_rate)
mask = (np.abs(freq) >= freq_band[0]) & (np.abs(freq) <= freq_band[1])
spectrum_filter = ft * mask
# подумать, нужно ли сохранять фазу
# plt.plot(freq[:int(len(freq)/2)], np.real(ft[:int(len(freq)/2)]))
# plt.plot(freq[:int(len(freq)/2)], np.real(spectrum_filter[:int(len(freq)/2)]))
signal_filter = ifft(spectrum_filter)
# plt.plot(SPL)
# plt.plot(np.real(signal_filter))

Запись аудио для прослушивания:

In [ ]:
# from scipy.io.wavfile import write
# write("example.wav", sample_rate, SPL.astype(np.int16))